In [8]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import optimizers
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
import tensorflow as tf
from keras.layers import BatchNormalization

In [2]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [3]:
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)


In [4]:
train_images = train_images.reshape((60000, 28 * 28))
test_images = test_images.reshape((10000, 28 * 28))
print(train_images.shape)
print(test_images.shape)

(60000, 784)
(10000, 784)


In [5]:
# 정규화
train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255
#print("train_image : ", train_images[:1])
#print("test_images : ", test_images[:1])

In [6]:
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)
print("train_labels : ", train_labels[0])
print("test_labels  : ", test_labels[0])

train_labels :  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
test_labels  :  [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [18]:
model = Sequential()
model.add(Dense(512, input_shape=(28 * 28,), activation='relu', kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu', kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu', kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu', kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 512)               401920    
                                                                 
 batch_normalization_12 (Ba  (None, 512)               2048      
 tchNormalization)                                               
                                                                 
 dropout_12 (Dropout)        (None, 512)               0         
                                                                 
 dense_16 (Dense)            (None, 512)               262656    
                                                                 
 batch_normalization_13 (Ba  (None, 512)               2048      
 tchNormalization)                                               
                                                                 
 dropout_13 (Dropout)        (None, 512)              

In [19]:
# 모델 컴파일 하기
model_checkpoint = ModelCheckpoint(filepath =  "./model.h5", monitor='val_loss', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 3)

# 모델 학습과정 설정하기
model.compile(loss = 'categorical_crossentropy', optimizer = optimizers.Adam(lr = 0.001), metrics=['accuracy'])

In [20]:
# fit() 메서드로 모델 훈련 시키기
model.fit(train_images, train_labels,
          epochs = 100, batch_size = 128,
          validation_data=(test_images, test_labels),
          callbacks = [model_checkpoint, early_stopping])

Epoch 1/100
464/469 [============================>.] - ETA: 0s - loss: 0.6163 - accuracy: 0.8161
Epoch 1: val_loss improved from inf to 0.16434, saving model to ./model.h5
469/469 [==============================] - 6s 7ms/step - loss: 0.6139 - accuracy: 0.8169 - val_loss: 0.1643 - val_accuracy: 0.9465
Epoch 2/100
464/469 [============================>.] - ETA: 0s - loss: 0.2600 - accuracy: 0.9220
Epoch 2: val_loss improved from 0.16434 to 0.12111, saving model to ./model.h5
469/469 [==============================] - 4s 9ms/step - loss: 0.2597 - accuracy: 0.9221 - val_loss: 0.1211 - val_accuracy: 0.9625
Epoch 3/100
469/469 [==============================] - ETA: 0s - loss: 0.1955 - accuracy: 0.9402
Epoch 3: val_loss improved from 0.12111 to 0.10361, saving model to ./model.h5
469/469 [==============================] - 3s 7ms/step - loss: 0.1955 - accuracy: 0.9402 - val_loss: 0.1036 - val_accuracy: 0.9686
Epoch 4/100
464/469 [============================>.] - ETA: 0s - loss: 0.1667 - acc

In [21]:
# 테스트 데이터로 정확도 측정하기
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('test_acc: ', test_acc)

313/313 [==============================] - 1s 3ms/step - loss: 0.0572 - accuracy: 0.9840
test_acc:  0.984000027179718


In [ ]:
predit = model.predict(test_images)
#for i in range(0,len(predit)) :
#    print("예측값 : ", predit[i], "\t실제값 : ", test_Y[i])
print(predit[0:5])
print(test_labels[0:5])

313/313 [==============================] - 1s 2ms/step
[[1.1174517e-07 7.8312860e-07 2.7569913e-06 3.1605412e-06 1.6404556e-06
  6.4888053e-07 1.6710008e-08 9.9998569e-01 1.7550484e-07 5.0359959e-06]
 [6.4647361e-07 4.6969112e-07 9.9999082e-01 7.0037763e-06 6.4728485e-09
  1.8170319e-08 2.9715375e-07 3.5059483e-07 4.0195047e-07 1.5360227e-10]
 [4.4830387e-07 9.9998128e-01 1.0595743e-06 5.6616919e-08 9.6427038e-07
  8.0534471e-07 1.6524996e-06 4.1250109e-06 9.4702546e-06 1.4202116e-07]
 [9.9993098e-01 4.0760887e-07 7.2272251e-06 7.3599222e-06 3.1464565e-06
  1.8885104e-06 3.5365414e-05 6.5613222e-07 1.8231093e-06 1.1244718e-05]
 [3.1645125e-06 5.1375105e-06 5.6902750e-06 2.1411377e-07 9.9929428e-01
  2.2474044e-06 3.2015614e-06 9.8246119e-06 1.3899928e-06 6.7486300e-04]]
[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


In [ ]:
predit = model.predict(test_images)
#for i in range(0,len(predit)) :
#    print("예측값 : ", predit[i], "\t실제값 : ", test_Y[i])
print(predit[0:5])
print(test_labels[0:5])

313/313 [==============================] - 1s 3ms/step
[[1.1174517e-07 7.8312860e-07 2.7569913e-06 3.1605412e-06 1.6404556e-06
  6.4888053e-07 1.6710008e-08 9.9998569e-01 1.7550484e-07 5.0359959e-06]
 [6.4647361e-07 4.6969112e-07 9.9999082e-01 7.0037763e-06 6.4728485e-09
  1.8170319e-08 2.9715375e-07 3.5059483e-07 4.0195047e-07 1.5360227e-10]
 [4.4830387e-07 9.9998128e-01 1.0595743e-06 5.6616919e-08 9.6427038e-07
  8.0534471e-07 1.6524996e-06 4.1250109e-06 9.4702546e-06 1.4202116e-07]
 [9.9993098e-01 4.0760887e-07 7.2272251e-06 7.3599222e-06 3.1464565e-06
  1.8885104e-06 3.5365414e-05 6.5613222e-07 1.8231093e-06 1.1244718e-05]
 [3.1645125e-06 5.1375105e-06 5.6902750e-06 2.1411377e-07 9.9929428e-01
  2.2474044e-06 3.2015614e-06 9.8246119e-06 1.3899928e-06 6.7486300e-04]]
[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]
